In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import json
import pandas as pd
import sys
import difflib
sys.path.append("../")
from utils import fidelity_check

In [3]:
screenshot_similarity = json.load(open("../record_replay/screenshots/onload_screenshot_similarity.json", 'r'))
screenshot_similarity = {s['directory']: s['screenshot_similarity'] for s in screenshot_similarity for s in screenshot_similarity}

In [6]:
def check_fidelity(dirr):
    screenshot_simi = screenshot_similarity[dirr.split('/')[-1]]
    live_html = open(f"{dirr}/live.html").read()
    archive_html = open(f"{dirr}/archive.html").read()
    live_element = json.load(open(f"{dirr}/live_elements.json"))
    archive_element = json.load(open(f"{dirr}/archive_elements.json"))
    my_fidelity = fidelity_check.verify(live_html, live_element, archive_html, archive_element)
    print(screenshot_simi >= 1, my_fidelity)

# check_fidelity('testcases/theftaz.azag.gov_1')

#### Test difflib

In [27]:
e1 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e2 = fidelity_check.htmlElement({'xpath': '/a/b', 'text': 'hello'})
e1 == e2

/a/b /a/b True


True

## Check archive differences

In [64]:
dirr = 'testcases/www.slideshare.net_1'

live_element = json.load(open(f"{dirr}/live_elements.json"))
archive_element = json.load(open(f"{dirr}/archive_elements.json"))
# fidelity_check.verify( live_element, archive_element)
live_unique, archive_unique = fidelity_check.diff(live_element, archive_element)
print(len(live_unique), len(archive_unique))
for i in live_unique:
    print("live:\n", i)
for i in archive_unique:
    print("archive:\n", i)

2 0
live:
   <div role="dialog" id="0d65c1f7-d305-4e2f-a572-5cbc31c158a3" aria-label="Cookie Consent Banner" aria-describedby="0d65c1f7-d305-4e2f-a572-5cbc31c158a3__label" class="osano-cm-window__dialog osano-cm-dialog osano-cm-dialog--position_bottom-right osano-cm-dialog--type_box">
    <button class=" osano-cm-dialog__close osano-cm-close ">
      <svg width="20px" height="20px" viewBox="0 0 20 20" role="img" aria-labelledby="419aab7f-534c-4264-a1fc-ed2d3ac7a889">
        <line role="presentation" x1="2" y1="2" x2="18" y2="18">
        <line role="presentation" x1="2" y1="18" x2="18" y2="2">
    <div class=" osano-cm-dialog__content osano-cm-content ">
      <a rel="noopener" tabindex="0" href="https://www.slideshare.net/privacy" target="_blank" class=" osano-cm-storage-policy osano-cm-content__link osano-cm-link " aria-describedby="osano-cm-aria.newWindow">
        Cookie Policy
      <span id="0d65c1f7-d305-4e2f-a572-5cbc31c158a3__label" class=" osano-cm-content__message osano-cm-